# 3.3 线性回归的简洁实现
## 参考资料
[1][PyTorch中文文档 torch.utils.data](https://pytorch-cn.readthedocs.io/zh/latest/package_references/data/)
## 相关知识
Python中super的概念  
nn.Linear  
y.view(-1, 1)

In [1]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.4.0


## 3.3.1 生成数据集

In [2]:
num_inputs = 2
num_examples = 2000
true_w = [3, -0.4]
true_b = 3.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

## 3.3.2 读取数据
Q：如何将特征和标签组合成训练集？  
A：使用`Data.TensorDataset()`API。

In [3]:
import torch.utils.data as Data

batch_size = 5

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
print(dataset[0])

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,            # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=8,              # 多线程来读数据,这里使用8个线程加载数据
)

(tensor([-0.6293,  1.1622]), tensor(0.8513))


In [4]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[ 0.2113, -1.0137],
        [-0.4737,  1.0359],
        [-0.7311,  0.6950],
        [-0.6162, -0.3579],
        [-1.4676, -0.0366]]) 
 tensor([ 4.2391,  1.3632,  0.7320,  1.4903, -1.2054])


## 3.3.3 定义模型

In [5]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [6]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )
print(net)
print(net[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [7]:
# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......
print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [8]:
# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [9]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.6445, -0.5849]], requires_grad=True)
Parameter containing:
tensor([0.1663], requires_grad=True)


## 3.3.4 初始化模型参数

In [10]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [11]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0023,  0.0045]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [12]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [13]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.04)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.04
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [14]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [15]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [16]:
num_epochs = 5
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000200
epoch 2, loss: 0.000096
epoch 3, loss: 0.000085
epoch 4, loss: 0.000063
epoch 5, loss: 0.000049


In [17]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[3, -0.4] tensor([[ 3.0011, -0.4005]])
3.2 tensor([3.2006])
